In [1]:
# common packages
import pandas as pd
import time

# # DL framework
import torch
from torchtext import data

from attractivedata import AttractiveData
from trainer import AttractiveTrainer

# TODO: 
- [ ] change save epoch to > 30
- [ ] ensemble all data

## Load and prepare data

In [2]:
train_file = 'data/train.csv'
val_file = 'example/val.csv'
test_file = 'data/test.csv'
pretrained_file = 'glove.840B.300d'
config = {
    'max_seq': 40,
    'min_freq': 0,
    'batch_size': 64,
    'pretrained_file': pretrained_file
}


In [3]:
AttractiveData = AttractiveData(train_file, val_file, test_file, pretrained_file, config)

In [4]:
# for i, sentence in enumerate(AttractiveData.test_data):
#     if i == 3:
#         print(vars(AttractiveData.train_data[i]), vars(sentence))

In [5]:
len(AttractiveData.CATEGORIES_LABEL.vocab.freqs)

18

In [6]:
max_len = 0
a = AttractiveData.train_data
for i in range(len(a)):
    if len(a[i].Headline) >= max_len:
        max_len = len(a[i].Headline)
max_len

38

## Start to train

In [7]:
config['timestr'] = time.strftime("%Y%m%d-%H%M%S")
config['save_name'] = 'CNN_LSTM'
config['input_dim'] = len(AttractiveData.TEXT.vocab)
config['embedding_dim'] = 300
config['category_dim'] = len(AttractiveData.CATEGORIES_LABEL.vocab)
config['category_embedding_dim'] = 8
config['hidden_dim'] = 30
config['output_dim'] = 1
config['log_steps'] = 10
config['epochs'] = 200
config['lr'] = {
    'encoder': 1e-5,
    'embedding': 6e-6,
    'linear': 1e-5
}
config['num_layers'] = 1
config['kernel_size'] = 3
config['dropout'] = 0.1
config['train_len'] = AttractiveData.train_len
config['val_len'] = AttractiveData.val_len
config['test_len'] = AttractiveData.test_len

pretrained_embeddings = AttractiveData.TEXT.vocab.vectors
print(pretrained_embeddings.shape)

torch.Size([12699, 300])


In [8]:
AttractiveTrainer = AttractiveTrainer(config, AttractiveData.device, AttractiveData.trainloader, AttractiveData.valloader, pretrained_embeddings)

In [9]:
AttractiveTrainer.model, AttractiveTrainer.config['total_params'], AttractiveTrainer.config['total_learned_params']

(AttractiveNet(
   (embedding): AttractiveEmbedding(
     (token): TokenEmbedding(12699, 300, padding_idx=1)
   )
   (bigramcnn): Sequential(
     (0): Conv1d(300, 200, kernel_size=(3,), stride=(1,), padding=(1,))
     (1): ReLU()
     (2): Conv1d(200, 100, kernel_size=(3,), stride=(1,), padding=(1,))
     (3): ReLU()
   )
   (trigramcnn): Sequential(
     (0): Conv1d(300, 200, kernel_size=(3,), stride=(1,), padding=(1,))
     (1): ReLU()
     (2): Conv1d(200, 100, kernel_size=(3,), stride=(1,), padding=(1,))
     (3): ReLU()
   )
   (encoder_bigram): LSTM(100, 30, batch_first=True, dropout=0.1, bidirectional=True)
   (encoder_trigram): LSTM(100, 30, batch_first=True, dropout=0.1, bidirectional=True)
   (linear): Sequential(
     (0): Linear(in_features=120, out_features=30, bias=True)
     (1): ReLU()
     (2): Linear(in_features=30, out_features=1, bias=True)
   )
 ),
 4357321,
 4357321)

In [10]:
AttractiveTrainer.train()

 28%|██▊       | 57/200 [00:22<00:56,  2.55it/s]
EP_train | train loss: 0.41313438509024825 | val loss: 0.3396399669787463 |
====
Epoch:  29%|██▉       | 58/200 [00:22<00:55,  2.55it/s]
EP_train | train loss: 0.4106382725285549 | val loss: 0.3377929943449357 |
====
Epoch:  30%|██▉       | 59/200 [00:23<00:55,  2.56it/s]
EP_train | train loss: 0.4092749343198888 | val loss: 0.3396182656288147 |
====
Epoch:  30%|███       | 60/200 [00:23<00:54,  2.57it/s]
EP_train | train loss: 0.4065582939222747 | val loss: 0.33435513751179563 |
====
Epoch:  30%|███       | 61/200 [00:24<00:54,  2.54it/s]
EP_train | train loss: 0.40418132333194506 | val loss: 0.3336371475575017 |
====
Epoch:  31%|███       | 62/200 [00:24<00:54,  2.55it/s]
EP_train | train loss: 0.40233098058139577 | val loss: 0.33080122809784085 |
====
Epoch:  32%|███▏      | 63/200 [00:24<00:53,  2.56it/s]
EP_train | train loss: 0.39976469395207426 | val loss: 0.3291642998947817 |
====
Epoch:  32%|███▏      | 64/200 [00:25<00:53,  2.5

## for classification, not better

## Below is testing

In [8]:
from attractivenet import AttractiveNet

# PATH = './model/CNN_LSTM_20201104-114114_0.3542.80'
# PATH = './model/CNN_LSTM_20201104-113908_0.3524.85'
# PATH = './model/CNN_LSTM_20201104-113437_0.3395.90'
# PATH = './model/CNN_LSTM_20201104-113008_0.3413.95'
# PATH = './model/CNN_LSTM_20201104-112639_0.3370.80'
# PATH = './model/CNN_LSTM_20201104-112044_0.3419.80'
# PATH = './model/CNN_LSTM_20201104-111439_0.3427.90'


# first time
# PATH = './model/CNN_LSTM_20201103-181740_0.4408.50'
# PATH = './model/CNN_LSTM_20201103-181044_0.4284.50'
# PATH = './model/CNN_LSTM_20201103-174348_0.4127.70'
# PATH = './model/CNN_LSTM_20201103-163042_0.4105.65'
# PATH = './model/CNN_LSTM_20201103-160026_0.4157.65'
# PATH = './model/CNN_LSTM_20201103-150358_0.3843.80'

load_model = AttractiveNet(config).to(AttractiveData.device)
load_model.load_state_dict(torch.load(PATH))
load_model.eval()

AttractiveNet(
  (embedding): AttractiveEmbedding(
    (token): TokenEmbedding(12699, 300, padding_idx=1)
  )
  (bigramcnn): Sequential(
    (0): Conv1d(300, 200, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): ReLU()
    (2): Conv1d(200, 100, kernel_size=(3,), stride=(1,), padding=(1,))
    (3): ReLU()
  )
  (trigramcnn): Sequential(
    (0): Conv1d(300, 200, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): ReLU()
    (2): Conv1d(200, 100, kernel_size=(3,), stride=(1,), padding=(1,))
    (3): ReLU()
  )
  (encoder_bigram): LSTM(100, 30, batch_first=True, dropout=0.1, bidirectional=True)
  (encoder_trigram): LSTM(100, 30, batch_first=True, dropout=0.1, bidirectional=True)
  (linear): Sequential(
    (0): Linear(in_features=120, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=1, bias=True)
  )
)

In [9]:
def predict_attractive(sentence, category, phase):
    indexed_sentence = [AttractiveData.TEXT.vocab.stoi[t] for t in sentence]
    indexed_category = [AttractiveData.CATEGORIES_LABEL.vocab.stoi[category]]
    tensor_sentence = torch.LongTensor(indexed_sentence).to(AttractiveData.device)
    tensor_category = torch.LongTensor(indexed_category).to(AttractiveData.device)

    tensor_sentence = tensor_sentence.unsqueeze(0)
    # print(tensor_sentence.shape)

    prediction = load_model(tensor_sentence, tensor_category, phase=phase)

    return prediction

In [10]:
# train mean = 3.15, test mean = 2.8
predict_list = []
with torch.no_grad():
    for i, sentence in enumerate(AttractiveData.test_data):
        prediction = predict_attractive(sentence.Headline, sentence.Category, 'test')
        predict_list.append(prediction.item())
        # predict_list.append(prediction.item())
AttractiveData.df_test['Label'] = predict_list
AttractiveData.df_test[['ID', 'Label']].to_csv(config['save_name'] + '.csv', index=False)

## Below for statistics

In [11]:
import statistics
from sklearn.metrics import mean_squared_error

In [12]:
# # train mean = 3.15, test mean = 2.8
# train_list = []
# for i, sentence in enumerate(AttractiveData.train_data):
#     prediction = predict_attractive(sentence.Headline, sentence.Category, 'train')
#     train_list.append(prediction.item())
#     # train_list.append(prediction.item())
# # print(train_list)
# mean_squared_error(pd.read_csv('data/train.csv').sort_values(['ID']).Label.to_list(), train_list), statistics.mean(train_list), statistics.stdev(train_list)

In [13]:
# train_list[0:5], pd.read_csv('data/train.csv').sort_values(['ID']).Label.to_list()[0:5]

In [14]:
# a = AttractiveData.df_train['Label'].to_list()
# statistics.mean(a), statistics.stdev(a)

In [15]:
statistics.mean(predict_list), statistics.stdev(predict_list)

(2.788740179087097, 0.34970232184253636)

In [16]:
baseline_list = pd.read_csv('baseline.csv').sort_values(['ID']).Label.to_list()
mean_squared_error(baseline_list, predict_list), statistics.mean(baseline_list), statistics.stdev(baseline_list)

(0.03214570970558876, 2.864683129188773, 0.37953271634933555)

In [11]:
mean_squared_error(pd.read_csv('baseline.csv').sort_values(['ID']).Label.to_list(), pd.read_csv('./predict/ensemble.csv').sort_values(['ID']).Label.to_list())

5.391702969068492e-05